In [1]:
import pandas as pd
from pprint import pprint
import ast
import numpy as np

# match data 전처리

In [ ]:
match_df = pd.read_csv('match_data_version1.csv')
match_df.shape

### 원본 데이터 확인

In [ ]:
list(match_df.columns)

In [ ]:
match_df.head()

### 필요없는 컬럼 제외

In [ ]:
match_df = match_df.drop(['Unnamed: 0','gameCreation','gameMode','gameType','mapId','platformId','seasonId','status.message','status.status_code'], axis=1)

In [ ]:
match_df.head()

### 중복 데이터 제거

In [ ]:
match_df = match_df.drop_duplicates(['gameId'])

### 5v5 랭크게임 및 Reset 게임 제외

In [ ]:
match_df = match_df[(match_df["queueId"] == 420) & (match_df["gameDuration"] > 240)]  # queueId : 420 -> 5v5 Ranked games
                                                                                        # https://static.developer.riotgames.com/docs/lol/queues.json 참고
                                                                                      # 게임시간 3분( + 게임재시작 투표시간 1분) 이전에 종료된 게임 제외0  

### Dictionary 타입 데이터 확인

In [ ]:
pprint(eval(match_df['participantIdentities'][1]))

In [ ]:
pprint(eval(match_df['participants'][1]))

### nested dictionary를 위한 재귀함수

In [ ]:
def solve_dict_column(dict_to_iterate, upper_key=None, pID = None):            # 하드 코딩, 추후 수정 필요
    global column_list
    global value_list
    for key, value in dict_to_iterate.items():
        if key == 'participantId':
            pID = 'P' + str(value)
        
        if type(value) == dict:
            solve_dict_column(value, key, pID)
        else:
            if upper_key == None:
                column_list.append(pID+ '_' + key)

            else :
                column_list.append(pID + '_' + str(upper_key) + '_' + key)
            value_list.append(value)


# 메모리 효율적으로 쓰게 수정 필요

### match data 의 participants, participantIdentities 칼럼의 nested dictionary to Dataframe (string)

In [ ]:
resolved = pd.DataFrame()
n = match_df.shape[0]
except_list = []

for i in range(n):
    temp = pd.DataFrame()
    column_list = []
    value_list = []
    if (len(eval(match_df.iloc[i]['participants'])) ==10) and  (len(eval(match_df.iloc[i]['participantIdentities'])) == 10):
        for x,y in enumerate([0,1,2,5]):
            column_list.append(match_df.columns[y])
            value_list.append(match_df.loc[i][y])
        for j in range(10):
            solve_dict_column(ast.literal_eval(match_df['participants'][i])[j])
            solve_dict_column(ast.literal_eval(match_df['participantIdentities'][i])[j])

        d=[]
        new_d = [] # 중복된 원소만 넣는 리스트
        for k in column_list:
            if k not in d: # 처음 등장한 원소
                d.append(k)
            else:
                if k not in new_d: # 이미 중복 원소로 판정된 경우는 제외
                    new_d.append(k)

        new_d # [1, 2] # 2회 이상 등장한 값들만 담긴 리스트
        
        temp = pd.DataFrame(data=[value_list], columns=column_list)
        temp = temp.drop(new_d, axis=1)

        resolved = pd.concat([resolved, temp], axis=0)
    else:
        except_list.append(i)

    if (i%1000) == 0:
        print(i)


resolved.head()

### 중간 저장

In [ ]:
resolved.to_csv('resolved.csv')

### Dict 변환 후 필요없는 컬럼 제거

In [ ]:
col_name=resolved.columns.tolist()
col_name

In [ ]:
search = "Diff"

removelist=["Diff","player", "Player", "perk", "highestAchievedSeasonTier", "totalScoreRank", "timeline_participantId"]

for i in range(len(removelist)):
    search = removelist[i]

    for word in col_name:
        if search in word: 
            print('>> remove: ' + word)
            resolved=resolved.drop(word, axis=1)

### Ranked Match Game 파일 저장

In [ ]:
resolved.to_csv('ranked_game.csv')

In [ ]:
# del resolved
# del match_df

# winner data

In [ ]:
winner_data = pd.read_csv('match_winner_data_version1.csv')
winner_data.shape

In [ ]:
list(winner_data.columns)

In [ ]:
winner_data.head()

### 중복 데이터 제거

In [ ]:
winner_data = winner_data.drop_duplicates(['gameId'])

### 밴픽 Dictionary solve

In [ ]:
for i in range(5):
    winner_data['ban_champion_'+ str(i+1)] = np.NaN

In [ ]:
winner_data.head()

In [ ]:
n = winner_data.shape[0]

for i in range(n): 
    try:
        for j in range(5):
            for key, value in ast.literal_eval(winner_data['bans'][i])[j].items():
                
                if key == 'championId':
                    winner_data.loc[i,'ban_champion_'+ str(j+1)] = value
    except:
        pass
                
winner_data.head()

In [ ]:
winner_data = winner_data.drop('bans', axis=1)

In [ ]:
winner_data.to_csv("winner_edit.csv", mode='w')

In [ ]:
# del winner_data

# Loser data

In [ ]:
loser_data = pd.read_csv('match_loser_data_version1.csv')
loser_data.shape

In [ ]:
list(loser_data.columns)

In [ ]:
loser_data.head()

### 중복 데이터 제거

In [ ]:
loser_data = loser_data.drop_duplicates(['gameId'])

### 밴픽 Dictionary solve

In [ ]:
for i in range(5):
    loser_data['ban_champion_'+ str(i+1)] = np.NaN
loser_data.head()

In [ ]:
n = loser_data.shape[0]

for i in range(n): 
    try:
        for j in range(5):
            for key, value in eval(loser_data['bans'][i])[j].items():
                
                if key == 'championId':
                    loser_data.loc[i,'ban_champion_'+ str(j+1)] = value
    except:
        pass
                
loser_data.head()

In [ ]:
loser_data = loser_data.drop('bans', axis=1)

In [ ]:
loser_data.to_csv("loser_edit.csv", mode='w')

In [ ]:
# del loser_data

# win/lose data에서 랭크게임 가져오기

### winner/loser data의 랭크게임을 추출하기 위해 전체 랭크게임의 gameId와 비교

In [ ]:
game_id_list = list(resolved["gameId"])

game_id_list

### 전체 랭크 게임과 gameId가 일치하는 경기들만 추출

In [ ]:
win_data_rank = winner_data[winner_data['gameId'].isin(game_id_list)]
lose_data_rank = loser_data[loser_data['gameId'].isin(game_id_list)]

In [ ]:
win_data_rank.to_csv("ranked_winner_game_edit.csv", index = None)
lose_data_rank.to_csv("ranked_loser_game_edit.csv", index = None)

### team100 데이터 추출

In [ ]:
team100 = resolved.copy()

resolved_col_name = resolved.columns.tolist()

removelist2=["P6", "P7", "P8", "P9", "P10"]

for i in range(len(removelist2)):
    search = removelist2[i]

    for word in resolved_col_name:
        if search in word: 
            print('>> remove: ' + word)
            team100 = team100.drop(word, axis=1)

team100.to_csv("team100_ranked.csv", index = None)

### team200 데이터 추출

In [ ]:
team200 = resolved.copy()

team100_columns = team100.columns.tolist()

del team100_columns[0:4]

team200 = team200.drop(team100_columns, axis=1)

team200.to_csv("team200_ranked.csv", index = None)